In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

In [3]:
from PY_tweet.lists_file import dumpJoblib, loadJoblib

data = loadJoblib('../DATA_all_texts/only_4labels_textdata.joblib')
label = loadJoblib('../DATA_all_texts/only_4labels_label.joblib')

In [4]:
from PY_preprocessing import remove

rm = remove.Remove()
data = rm.url(data)
data[10:]

['新型コロナウイルス(2019-nCoV, COVID-19)の重症管理に役立つコクランエビデンスを集めた特集号の日本語訳が完了しました。\nそれぞれのコクランレビューのリンクは訳のあるものについては日本語表記されるリンクにしています… ',
 '検疫官も感染してしまうとなると、やはり乗客と接する乗員は全数検査するべきだと思う。\n\n #新型肺炎 #新型コロナウィルス #COVID19 #クルーズ船 #ひるおび',
 '新型コロナウイルス（COVID－19）治療の最前線に立つ、大曲貴夫先生にお話を伺いました。「国内では一人も死なせない」と連日奮闘なさって、体重が2キロ落ちたとのこと。院内感染管理の重要性や最新治療、一般の人ができることも話して頂き… ',
 'クルーズ船「ダイヤモンドプリンス号」の乗客輸送支援として #小牧基地 #第１輸送航空隊 管理隊員４名と大型バス２台が出発しました。\n#新型コロナウイルス \n#災害派遣 \n#covid19 ',
 '急に新型コロナウイルスCOVID-19患者数が1日で14840例（うち12000例近くは武漢）急増しましたが、これはこれまでCTやPCRで陽性とでなかったけど疑わしい症例を臨床診断として患者数に含めたからのようです。PCRの検査感… ',
 '都内のタクシー運転手が感染。遅かれ早かれ来ると判っていた事だが…。情報の確実さの責任に束縛された専門家の発信が後手に回った結果、世の中全体の、そして業界の動きが遅きに失した感がある。\n#NHK #首都圏ネットワーク #新型コロナウイルス #COVID19 #感染症',
 '【速報】東京都内のタクシー運転手の男性、新型コロナウイルスに感染確認\n#COVID19 #新型肺炎 #新型コロナウイルス',
 '中国では看護師はウィルスが髪につかないようにマルコメちゃんに。\n髪の長い女性は切ったりまとめた方が良いかもしれない。\n\n#coronavirus\n#コロナウィルス\n#新型コロナウィルス\n#COVID2019 ',
 '#新型コロナウイルス\u3000COVID-19\n\n和歌山県の感染者は\u3000医師とのことです。\n\n医師も命懸けです！\n\n正しい #感染対策 をお願いします。\n\n',
 '日本政府､緊急ウイルス対策に総額153億円\u30

In [5]:
!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0 pytorch-lightning==1.2.7

In [6]:
import random 
import glob
import json
from tqdm import tqdm
import unicodedata

import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from transformers import BertTokenizer, BertModel

from PY_classification.multi_classification import BertForSequenceClassificationMultiLabel

MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

In [7]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [8]:
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
bert_scml = BertForSequenceClassificationMultiLabel(
    MODEL_NAME, num_labels=4
) 
bert_scml = bert_scml.to(device=device)

In [9]:
labels = []
for emotion in label:
    if emotion == 63:
        labels.append([1, 0, 0, 0])
    elif emotion == 64:
        labels.append([0, 1, 0, 0])
    elif emotion == 65:
        labels.append([0, 0, 1, 0])
    else:
        labels.append([0, 0, 0, 1])

In [10]:
max_length = 152
dataset_for_loader = []

for text, label in zip(data, labels):
    encoding = tokenizer(
      text, 
      max_length=max_length,
      padding='max_length',
      truncation=True
    )
    encoding['labels'] = label
    encoding = {k: torch.tensor(v).to(device=device) for k, v in encoding.items() }
    dataset_for_loader.append(encoding)
    

random.shuffle(dataset_for_loader)
n = len(dataset_for_loader)
n_train = int(0.6*n)
n_val = int(0.2*n)
dataset_train = dataset_for_loader[:n_train]
dataset_val = dataset_for_loader[n_train:n_train+n_val]
dataset_test = dataset_for_loader[n_train+n_val:]

In [11]:
dataloader_train = DataLoader(
    dataset_train, batch_size=16, shuffle=True
)
dataloader_val = DataLoader(
    dataset_val, batch_size=16
)
dataloader_test = DataLoader(
    dataset_test, batch_size=16
)

dataloader_val

In [12]:
from PY_classification.pytorch_lightning_multi import BertForSequenceClassificationMultiLabel_pl

checkpoint = pl.callbacks.ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    save_top_k=1,
    save_weights_only=True,
    dirpath='part/model/'
)

#学習方法指定
trainer = pl.Trainer(
    gpus=1,
    max_epochs=10,
    callbacks=[checkpoint]
)

model = BertForSequenceClassificationMultiLabel_pl(
    MODEL_NAME, num_labels=4, lr=1e-5
)

trainer.fit(model, dataloader_train, dataloader_val)

/home/takakiyuto/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: Checkpoint directory part/model/ exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                                    | Params
----------------------------------------------------------------------
0 | bert_scml | BertForSequenceClassificationMultiLabel | 110 M 
----------------------------------------------------------------------
110 M     Trainable params
0         Non-trainable params
110 M     Total params
442.482   Total estimated model params size (MB)
/home/takakiyuto/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 whic

Validation sanity check: 0it [00:00, ?it/s]

/home/takakiyuto/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

1

In [13]:
best_model_path = checkpoint.best_model_path
print(f'validation data loss:{checkpoint.best_model_score}')
print(f'The best path is {best_model_path}')

validation data loss:0.25791460275650024
The best path is /home/takakiyuto/Desktop/RESEARCH-COVID19-datasets/IPY_model/part/model/epoch=1-step=2813-v2.ckpt


In [14]:
%load_ext tensorboard
%tensorboard --logdir ./

In [15]:
test = trainer.test(test_dataloaders=dataloader_test, ckpt_path=best_model_path)
print(f'Accuracy: {test[0]["accuracy"]:.4f}')

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/takakiyuto/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:52: UserWarning: The dataloader, test dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'accuracy': 0.7757196426391602}
--------------------------------------------------------------------------------
Accuracy: 0.7757
